In [1]:
import requests
import pandas as pd
from pandas import json_normalize
import json

ticker  = 'AMZN'

api = 'sandbox'
token  = 'Tpk_8d2468bfcd924cd69d65c72317925e57'
period = 'annual'
last  = '3'

## Fetching Information

In [2]:
# fetching balance sheet
response = requests.get('https://'+api+'.iexapis.com/stable/stock/'+ticker+'/balance-sheet?period='+period+'&last='+last+'&token='+token)
balance = json_normalize(response.json(), record_path=['balancesheet'])

# fetching income and cashflow statements
response = requests.get('https://'+api+'.iexapis.com/stable/stock/'+ticker+'/income?period='+period+'&last='+last+'&token='+token)
income = json_normalize(response.json(), record_path=['income'])

response = requests.get('https://'+api+'.iexapis.com/stable/stock/'+ticker+'/cash-flow?period='+period+'&last='+last+'&token='+token)
cashflow = json_normalize(response.json(), record_path=['cashflow'])

# fetching shares issued
response = requests.get('https://'+api+'.iexapis.com/stable/stock/'+ticker+'/stats/sharesOutstanding?token='+token)
shares = response.json()

response = requests.get('https://'+api+'.iexapis.com/stable/time-series/REPORTED_FINANCIALS/'+ticker+'/10-K?token='+token)
financials = json_normalize(response.json())

KeyboardInterrupt: 

# Calculating of F_score

In [ ]:
f_score  = 0

f_score += (income.iloc[0]['netIncome']/balance.iloc[1]['totalAssets'] > 0)     # Asset Returns

curr = income.iloc[0]['netIncome']/balance.iloc[1]['totalAssets']
prev = income.iloc[1]['netIncome']/balance.iloc[2]['totalAssets']

f_score += curr>prev    # Change in Assets Returns

f_score += cashflow.iloc[0]['cashFlow']>0     # Operating Cashflow

a_cash = cashflow.iloc[0]['cashFlow']/balance.iloc[1]['totalAssets']
a_income = income.iloc[0]['netIncome']/balance.iloc[1]['totalAssets']

f_score += a_cash > a_income      # Accurals

f_score += balance.iloc[0]['longTermDebt']/balance.iloc[0]['totalAssets'] < balance.iloc[1]['longTermDebt']/balance.iloc[1]['totalAssets']
      # Leverage Ratio 

f_score += balance.iloc[0]['currentAssets'] / balance.iloc[0]['totalCurrentLiabilities'] > balance.iloc[1]['currentAssets'] / balance.iloc[1]['totalCurrentLiabilities']
      # Current Ratio

f_score += shares < financials['CommonStockSharesIssued'].iloc[0]       # Number of Shares

f_score += income.iloc[0]['grossProfit']/income.iloc[0]['totalRevenue'] > income.iloc[1]['grossProfit']/income.iloc[1]['totalRevenue']
      # Change in Gross Margin

f_score += income.iloc[0]['totalRevenue']/balance.iloc[1]['totalAssets'] > income.iloc[1]['totalRevenue']/balance.iloc[2]['totalAssets']
      # Change in Asset Turnover Ratio 

# Final Piotroski F-Score
print(f_score)